In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import librosa
import zipfile
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
from sklearn.neighbors import KNeighborsClassifier

# Load metadata
metadata_path = '/content/drive/MyDrive/Bird sound classification data/bird_songs_metadata.csv'
zip_path = '/content/drive/MyDrive/Bird sound classification data/archive (1).zip'
metadata = pd.read_csv(metadata_path)

# Extract features from audio files
def extract_features(file_path, zip_ref):
    with zip_ref.open(file_path) as file:
        y, sr = librosa.load(file, sr=None)
        S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        S_DB = librosa.power_to_db(S, ref=np.max)
        return np.mean(S_DB, axis=1)  # Using mean as a simple feature

# Prepare dataset for Log Mel features
features = []
labels = []

with zipfile.ZipFile(zip_path, 'r') as z:
    for index, row in metadata.iterrows():
        file_path = os.path.join('wavfiles', row['filename'])
        if file_path in z.namelist():
            features.append(extract_features(file_path, z))
            labels.append(row['species'])

features = np.array(features)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Split the dataset for Log Mel features
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# Extract MFCC features
def extract_mfcc_features(file_path, zip_ref):
    with zip_ref.open(file_path) as file:
        y, sr = librosa.load(file, sr=None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        return np.mean(mfcc, axis=1)

# Prepare dataset for MFCC features
mfcc_features = []
labels_mfcc = []

with zipfile.ZipFile(zip_path, 'r') as z:
    for index, row in metadata.iterrows():
        file_path = os.path.join('wavfiles', row['filename'])
        if file_path in z.namelist():
            mfcc_features.append(extract_mfcc_features(file_path, z))
            labels_mfcc.append(row['species'])

mfcc_features = np.array(mfcc_features)
label_encoder_mfcc = LabelEncoder()
labels_encoded_mfcc = label_encoder_mfcc.fit_transform(labels_mfcc)

# Split the dataset for MFCC features
X_train_mfcc, X_test_mfcc, y_train_mfcc, y_test_mfcc = train_test_split(mfcc_features, labels_encoded_mfcc, test_size=0.2, random_state=42)

# Combine Log Mel and MFCC features
combined_features = np.hstack((features, mfcc_features))
labels_combined = labels_encoded  # Assuming the same labels

# Split the dataset for combined features
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(combined_features, labels_combined, test_size=0.2, random_state=42)


In [3]:
# Train and save KNN model for Log Mel features
def train_save_knn_log_mel(X_train, y_train, model_path='knn_log_mel.pkl'):
    knn_log_mel = KNeighborsClassifier(n_neighbors=5)
    knn_log_mel.fit(X_train, y_train)
    with open(model_path, 'wb') as f:
        pickle.dump(knn_log_mel, f)
    return knn_log_mel

# Train and save KNN model for MFCC features
def train_save_knn_mfcc(X_train_mfcc, y_train_mfcc, model_path='knn_mfcc.pkl'):
    knn_mfcc = KNeighborsClassifier(n_neighbors=5)
    knn_mfcc.fit(X_train_mfcc, y_train_mfcc)
    with open(model_path, 'wb') as f:
        pickle.dump(knn_mfcc, f)
    return knn_mfcc

# Train and save KNN model for combined Log Mel and MFCC features
def train_save_knn_combined(X_train_combined, y_train_combined, model_path='knn_combined.pkl'):
    knn_combined = KNeighborsClassifier(n_neighbors=5)
    knn_combined.fit(X_train_combined, y_train_combined)
    with open(model_path, 'wb') as f:
        pickle.dump(knn_combined, f)
    return knn_combined

# Example usage
knn_log_mel = train_save_knn_log_mel(X_train, y_train, 'knn_log_mel.pkl')
knn_mfcc = train_save_knn_mfcc(X_train_mfcc, y_train_mfcc, 'knn_mfcc.pkl')
knn_combined = train_save_knn_combined(X_train_combined, y_train_combined, 'knn_combined.pkl')


In [4]:
from sklearn.svm import SVC

# Train and save SVM model for Log Mel features
def train_save_svm_log_mel(X_train, y_train, model_path='svm_log_mel.pkl'):
    svm_log_mel = SVC(probability=True, random_state=42)
    svm_log_mel.fit(X_train, y_train)
    with open(model_path, 'wb') as f:
        pickle.dump(svm_log_mel, f)
    return svm_log_mel

# Train and save SVM model for MFCC features
def train_save_svm_mfcc(X_train_mfcc, y_train_mfcc, model_path='svm_mfcc.pkl'):
    svm_mfcc = SVC(probability=True, random_state=42)
    svm_mfcc.fit(X_train_mfcc, y_train_mfcc)
    with open(model_path, 'wb') as f:
        pickle.dump(svm_mfcc, f)
    return svm_mfcc

# Train and save SVM model for combined Log Mel and MFCC features
def train_save_svm_combined(X_train_combined, y_train_combined, model_path='svm_combined.pkl'):
    svm_combined = SVC(probability=True, random_state=42)
    svm_combined.fit(X_train_combined, y_train_combined)
    with open(model_path, 'wb') as f:
        pickle.dump(svm_combined, f)
    return svm_combined

# Example usage
svm_log_mel = train_save_svm_log_mel(X_train, y_train, 'svm_log_mel.pkl')
svm_mfcc = train_save_svm_mfcc(X_train_mfcc, y_train_mfcc, 'svm_mfcc.pkl')
svm_combined = train_save_svm_combined(X_train_combined, y_train_combined, 'svm_combined.pkl')


In [5]:
def load_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

# Example usage
knn_log_mel_loaded = load_model('knn_log_mel.pkl')
knn_mfcc_loaded = load_model('knn_mfcc.pkl')
knn_combined_loaded = load_model('knn_combined.pkl')
svm_log_mel_loaded = load_model('svm_log_mel.pkl')
svm_mfcc_loaded = load_model('svm_mfcc.pkl')
svm_combined_loaded = load_model('svm_combined.pkl')


In [6]:
import pickle

# Define the path to your model files
model_path_knn_log_mel = '/content/knn_log_mel.pkl'
model_path_knn_mfcc = '/content/knn_mfcc.pkl'
model_path_knn_combined = '/content/knn_combined.pkl'

model_path_svm_log_mel = '/content/svm_log_mel.pkl'
model_path_svm_mfcc = '/content/svm_mfcc.pkl'
model_path_svm_combined = '/content/svm_combined.pkl'

# Function to load a model
def load_model(model_path):
    with open(model_path, 'rb') as f:
        model = pickle.load(f)
    return model

# Load the models
knn_log_mel_loaded = load_model(model_path_knn_log_mel)
knn_mfcc_loaded = load_model(model_path_knn_mfcc)
knn_combined_loaded = load_model(model_path_knn_combined)

svm_log_mel_loaded = load_model(model_path_svm_log_mel)
svm_mfcc_loaded = load_model(model_path_svm_mfcc)
svm_combined_loaded = load_model(model_path_svm_combined)


In [7]:
# Example prediction using the loaded KNN Log Mel model
# X_test should be your test data
y_pred_knn_log_mel = knn_log_mel_loaded.predict(X_test)

# Example prediction using the loaded SVM MFCC model
# X_test_mfcc should be your test data for MFCC features
y_pred_svm_mfcc = svm_mfcc_loaded.predict(X_test_mfcc)

print(y_pred_knn_log_mel)
print(y_pred_svm_mfcc)


[1 1 2 ... 4 2 1]
[1 1 2 ... 4 0 1]


In [15]:
from google.colab import drive
import zipfile
import pickle
import os

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Extract the contents of the zip file
#zip_path = '/content/drive/MyDrive/Bird sound classification data/3CNN_1RNN_regularized_model.zip'
zip_path="/content/drive/MyDrive/Bird sound classification data/ZIP_KNN_and_SVM models.zip"
extract_path = '/content/ZIP_KNN_and_SVM models/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Step 3: List all files in the extracted directory
extracted_files = os.listdir(extract_path)
print("Extracted files:", extracted_files)

# Step 4: Function to load a pickle file
def load_pickle_file(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# Step 5: Load and inspect the pickle files
for file_name in extracted_files:
    if file_name.endswith('.pkl'):
        file_path = os.path.join(extract_path, file_name)
        loaded_data = load_pickle_file(file_path)
        print(f"Contents of {file_name}:")
        print(loaded_data)
        print("\n")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Extracted files: ['knn_mfcc.pkl', 'svm_combined.pkl', 'knn_log_mel.pkl', 'svm_log_mel.pkl', 'knn_combined.pkl', 'svm_mfcc.pkl', 'KNN and SVM models']
Contents of knn_mfcc.pkl:
KNeighborsClassifier()


Contents of svm_combined.pkl:
SVC(probability=True, random_state=42)


Contents of knn_log_mel.pkl:
KNeighborsClassifier()


Contents of svm_log_mel.pkl:
SVC(probability=True, random_state=42)


Contents of knn_combined.pkl:
KNeighborsClassifier()


Contents of svm_mfcc.pkl:
SVC(probability=True, random_state=42)


